In [19]:
import requests
import pandas as pd
import os
from pyspark.sql import SparkSession
from concurrent.futures import ThreadPoolExecutor, as_completed
import json
from dotenv import load_dotenv

In [20]:
load_dotenv()
TOKEN= os.getenv('TOKEN')


In [21]:
spark = SparkSession.builder.appName("API SP").getOrCreate()

In [22]:
key = TOKEN
pages = 5000

url = f'https://specieslink.net/ws/1.0/search?stateprovince=Alagoas&apikey={key}'


In [23]:
r = requests.get(f'{url}&offset=0&limit={pages}&apikey={key}')
r
#r.json()['features']

<Response [200]>

In [24]:
total_registros = r.json()['numberMatched']
total_registros

90265

In [25]:
total_json = []
page_init = 0

In [26]:
qtd_paginas = (total_registros // pages) + 1
qtd_paginas

19

In [27]:
sc = spark.sparkContext  # Obter o contexto Spark

In [28]:
with ThreadPoolExecutor(max_workers=10) as executor:
    futures = [executor.submit(requests.get, f'{url}&offset={page_init}&limit={pages}&apikey={key}') for page in range(1, qtd_paginas + 1)]

    for future in as_completed(futures):
        response = future.result()
        total_json.extend(response.json()['features'])

print(f"Total de registros obtidos: {len(total_json)}")

Total de registros obtidos: 95000


In [29]:
with open('all_records.json', 'w') as file:
    json.dump(total_json, file)